In [2]:
# -------------------------------
# 0. 라이브러리 및 환경 설정
# -------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from scipy.stats import zscore, pointbiserialr
from sklearn.preprocessing import PowerTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings("ignore")

# 그래프 한글폰트(선택) ─ macOS 예시
plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'Apple SD Gothic Neo' # 띄어쓰기 포함
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

In [5]:
kospi = pd.read_csv("../data/raw/코스피_상장폐지.csv")
kosdaq = pd.read_csv("../data/raw/코스닥_상장폐지.csv")

In [6]:
fail = pd.concat([kospi, kosdaq])

In [ ]:
fail.to_csv("../data/processed/fail.csv")

In [ ]:
flag = fail['종목코드'].unique()

bs = pd.read_csv("../data/processed/BS.csv")

bsset = set(bs['거래소코드'].unique())
flagset = set(flag)

print(f"BS에 있는 기업수: {len(bsset)}")
print(f"flag에 있는 기업수: {len(flagset)}")
print(f"BS에 있는 상장폐지 기업수: {len(bsset & flagset)}")
print(f"BS에 없는 상장폐지 기업수: {len(flagset - bsset)}")
print(f"최종 기업수: {len(bsset - flagset)}")


In [7]:
c_ratio = pd.read_csv("../data/raw/연결 재무비율(IFRS)_0613.csv")
n_ratio = pd.read_csv("../data/raw/재무비율(IFRS)_0613.csv")
c_bs = pd.read_csv("../data/raw/연결 재무제표(IFRS)_0613.csv")
n_bs = pd.read_csv("../data/raw/재무제표(IFRS)_0613.csv")
delisting = pd.concat([kospi, kosdaq])
df = pd.DataFrame()

In [ ]:
c_bs.columns.tolist()

In [ ]:
n_bs.columns.tolist()

In [ ]:
bs = pd.read_csv("../data/processed/BS.csv")
ratio = pd.read_csv("../data/processed/ratio.csv")

print(bs.shape)
print(ratio.shape)

df = pd.concat([bs, ratio], axis=1)
df.to_csv("../data/processed/bs_ratio.csv", index=False)

In [ ]:
df.columns

In [ ]:
df.columns.tolist()

In [ ]:
# -------------------------------
# 1. 데이터 불러오기
# -------------------------------
### TODO: 실제 파일 경로를 지정하세요
df = pd.read_csv("../data/processed/bs_ratio.csv")

# -------------------------------
# 2. 주요 변수 지정
# -------------------------------
### TODO: 컬럼명을 실제 데이터에 맞춰 수정
id_col      = '거래소코드'
keep_cols = [
    '[A100000000]자산(*)(IFRS연결)(천원)',                                    # 총자산
    '[A800000000]부채(*)(IFRS연결)(천원)',                                    # 총부채
    '[A600000000]자본(*)(IFRS연결)(천원)',                                    # 자본
    '[B420000000]* (정상)영업손익(보고서기재)(3개월)(IFRS연결)(천원)',     # 영업이익 (분기)
    '[B840000000]당기순이익(손실)(3개월)(IFRS연결)(천원)',                   # 순이익 (분기)
    '[D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)',    # 영업현금흐름
    '부채비율(IFRS연결)',                                                      # Debt Ratio
    '차입금의존도(IFRS연결)',                                                  # Borrowing Dependence
    '총자본회전률(IFRS연결)',                                                  # Asset Turnover
    'PBR(최저)(IFRS연결)'                                                      # PBR (Lowest)
]
df = df.drop(columns=[c for c in df.columns if c not in keep_cols])
df.columns = ['Asset', 'Debt', 'Equity', 'OperatingIncome', 'NetIncome', 'OperatingCashFlow', 'DebtRatio', 'BorrowingDependence', 'AssetTurnover', 'PBR']
cat_cols = []                  # 범주형 변수 있다면 추가
df.columns

In [ ]:
# 2-1. 정보 요약
display(df.info())
display(df.head())

# # 2-2. 중복 확인 (동일 기업·연도 중복행 탐지)
# dup_mask = df.duplicated(subset=[id_col, 'year'])
# print(f"중복 행 수: {dup_mask.sum()}")
# if dup_mask.any():
#     df = df[~dup_mask]  # 필요 시 제거

In [ ]:
# 3-1. 변수별 결측 비율
null_ratio = df.isna().mean().sort_values(ascending=False)
display(null_ratio.head(10))

# 3-2. 히트맵 시각화
plt.figure(figsize=(10,4))
msno.matrix(df.sample(min(1000, len(df_bs))))
plt.show()

In [ ]:
# 4-1. 기술통계
desc = df.describe().T
display(desc)

# 4-2. z-score 기반 이상치 마스크
z_scores = df.apply(zscore)
outlier_mask = (np.abs(z_scores) > 3).any(axis=1)
print(f"z>|3| 이상치 행 비율: {outlier_mask.mean():.2%}")

# 4-3. 상자그림(샘플 10개 변수)
numeric_cols = df.select_dtypes(include='number').columns

for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col], orient='h')
    plt.title(f"Boxplot of {col}")
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()

In [ ]:
# 6-1. 피어슨 상관행렬
corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
plt.title("Correlation Matrix"); plt.show()

# # 6-2. VIF 계산
# X = df.drop(columns=[], errors='ignore')  # 필요 시 제외 변수 지정
# vif_vals = pd.Series(
#     [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
#     index=X.columns, name='VIF'
# )
# display(vif_vals.sort_values(ascending=False).head(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# ─────────────────────────────────────────────
# QQ-Plot: 각 변수별로 화면에 표시
# ─────────────────────────────────────────────
for col in df.columns:
    sm.qqplot(df[col].dropna(), line='45', fit=True)
    plt.title(f"QQ Plot of {col}")
    plt.xlabel("Theoretical Quantiles")
    plt.ylabel("Sample Quantiles")
    plt.tight_layout()
    plt.show()
    plt.close()

# ─────────────────────────────────────────────
# Scatter: 'Asset' 기준으로 나머지 변수들과 화면에 표시
# ─────────────────────────────────────────────
for col in df.columns:
    if col == 'Asset':
        continue
    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=df['Asset'], y=df[col], alpha=0.6)
    plt.title(f"Asset vs {col}")
    plt.xlabel("Asset")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()
    plt.close()


In [8]:
industry = pd.read_excel("../data/raw/산업분류.xlsx")

In [9]:
industry["통계청 한국표준산업분류 11차(대분류)"].value_counts()

통계청 한국표준산업분류 11차(대분류)
제조업                         18764
정보통신업                        3552
도매 및 소매업                     2108
전문, 과학 및 기술 서비스업             2100
금융 및 보험업                     1482
건설업                           756
운수 및 창고업                      341
사업시설 관리, 사업 지원 및 임대 서비스업      248
전기, 가스, 증기 및 공기조절 공급업         164
교육 서비스업                       133
예술, 스포츠 및 여가관련 서비스업           129
부동산업                           67
숙박 및 음식점업                      58
농업, 임업 및 어업                    49
수도, 하수 및 폐기물 처리, 원료 재생업        36
협회 및 단체, 수리 및 기타 개인 서비스업       28
보건업 및 사회복지 서비스업                12
광업                              3
Name: count, dtype: int64

In [10]:
industry = industry[industry['통계청 한국표준산업분류 11차(대분류)'] != "금융 및 보험업"]
industry = industry[industry['통계청 한국표준산업분류 11차(대분류)'] != "부동산업"]
industry = industry[industry['통계청 한국표준산업분류 11차(대분류)'] != "전기, 가스, 증기 및 공기조절 공급업 (공공 유틸리티)"]
industry

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 11차(대분류),통계청 한국표준산업분류 코드 11차(대분류),상장협 산업분류(대분류),상장협 산업분류 코드(대분류)
12,(주)CMG제약,58820,2012/12,제조업,21,제조업,30000
13,(주)CMG제약,58820,2013/12,제조업,21,제조업,30000
14,(주)CMG제약,58820,2014/12,제조업,21,제조업,30000
15,(주)CMG제약,58820,2015/12,제조업,21,제조업,30000
16,(주)CMG제약,58820,2016/12,제조업,21,제조업,30000
...,...,...,...,...,...,...,...
30025,흥아해운(주),3280,2019/12,운수 및 창고업,50,운수 및 창고업,80000
30026,흥아해운(주),3280,2020/12,운수 및 창고업,50,운수 및 창고업,80000
30027,흥아해운(주),3280,2021/12,운수 및 창고업,50,운수 및 창고업,80000
30028,흥아해운(주),3280,2022/12,운수 및 창고업,50,운수 및 창고업,80000


In [ ]:
len(c_bs)

In [11]:
# industry에 존재하는 거래소코드 목록 추출
valid_codes = industry['거래소코드'].unique()

# 각 데이터프레임에서 valid_codes에 포함된 행만 남기기
c_bs     = c_bs    [c_bs    ['거래소코드'].isin(valid_codes)].reset_index(drop=True)
c_ratio  = c_ratio [c_ratio ['거래소코드'].isin(valid_codes)].reset_index(drop=True)
n_bs     = n_bs    [n_bs    ['거래소코드'].isin(valid_codes)].reset_index(drop=True)
n_ratio  = n_ratio [n_ratio ['거래소코드'].isin(valid_codes)].reset_index(drop=True)

In [12]:
c_bs.to_csv("../data/raw/연결재무제표.csv", index=False)
n_bs.to_csv("../data/raw/개별재무제표.csv", index=False)
c_ratio.to_csv("../data/raw/연결재무비율.csv", index=False)
n_ratio.to_csv("../data/raw/개별재무비율.csv", index=False)

In [19]:
bs = pd.read_csv("../data/processed/BS.csv")
ratio = pd.read_csv("../data/processed/ratio.csv")

df = pd.merge(bs, ratio, on=["회사명", "거래소코드", "회계년도"])
df.to_csv("../data/processed/BS_ratio.csv", index=False)

In [53]:
fail = pd.read_csv("../data/processed/fail.csv")

In [54]:
fail.columns

Index(['Unnamed: 0', '번호', '회사명', '종목코드', '폐지일자', '폐지사유', '비고'], dtype='object')

In [55]:
fail.drop(["Unnamed: 0", "번호", "비고"], axis=1, inplace=True)

In [57]:
fail_set = set(fail['폐지사유'].unique().tolist())

In [75]:
# ─────────────────────────────────────────────
# 1) “실적(재무·시장요건)‧부도”에 의한 상장폐지 사유
# ─────────────────────────────────────────────
performance_cols = [
    # ── 시장·재무 요건 미달 ──
    '시가총액 미달',
    '시가총액 40억원 미달',
    '보통주 주가수준미달',
    '보통주 주식분포요건 미충족 2년 계속',
    '매출액 미달(50억원 미만) 2년 계속',
    '자본전액잠식',
    '자본전액잠식 등',
    '자본전액잠식등',
    '자본잠식(자본금의 50/100이상 잠식) 2년 계속',
    '2반기 연속 자본잠식률 50% 이상',
    '2반기 연속 자본잠식률 50%이상',
    '2회 연속 자본잠식률 50%이상',
    '자본잠식률 50% 이상으로 관리종목 지정 이후 최근 반기의 재무제표에 대한 감사인의 감사의견거절 등',
    '최근 5사업연도 연속 영업손실 발생',
    '최근 5사업연도 연속 영업손실 발생 등',
    '최근3사업연도중2사업연도자기자본 50%초과법인세비용차감전손실발생',
    

    # ── 감사(意見거절·한정·부적정) ──
    '감사의견 의견거절 2년 계속',
    '감사의견 의견거절',
    '감사의견 부적정',
    '감사의견 거절',
    '감사의견 거절(감사범위 제한)',
    '감사의견 거절(감사범위 제한 및 계속기업가정 불확실성)',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력 불확실성)',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성',
    '감사의견 거절(감사범위 제한 및 기업회계기준 위배)',
    '감사의견 거절(내부회계관리제도상의 취약점, 계속기업가정의 불확실성 및 재고자산 관련 감사범위 제한)',
    '감사의견 거절(감사범위 제한으로 인한 의견거절)',
    '감사의견거절(감사범위 제한)',
    '감사의견거절(감사범위제한)',
    '감사의견거절(감사범위 제한 및 계속기업존속 불확실성)',
    '감사의견거절(감사범위제한 및 계속기업 존속 불확실)',
    '감사의견거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '감사의견거절(감사범위 제한 및 계속기업존속능력 불확실성)',
    '감사의견거절(범위제한 및 계속기업 존속능력에 대한 불확실성)',
    '감사의견거절(감사범위 제한 및 계속기업으로서의 존속능력에 대한 불확실성)',
    "'16사업연도 감사의견거절(계속기업 존속능력에 대한 불확실성) 및 '17사업연도 반기 감사의견거절(계속기업 존속능력에 대한 불확실성)",
    # "'16사업연도 감사의견거절(계속기업 존속능력에 대한 불확실성) 및 '17사업연도 반기 감사의견거절(계속기업 존속능력에 대한 불확실성)'",
    '감사범위제한으로 인한 한정의견',
    '감사의견 한정(감사범위 제한)',
    '의견거절(감사범위 제한)',
    '감사의견 거절(계속기업 존속능력에 대한 불확실성 및 감사범위 제한)',
    '감사의견 거절(계속기업 존속능력에 대한 불확실성)',
    '감사의견거절(계속기업으로서의 존속능력에 대한 불확실성)',
    '반기 재무제표에 대한 검토의견거절로 관리종목 지정후 자본잠식률 50% 이상',
    '감사 의견거절(감사범위 제한)',
    '감사의견 거절(감사범위 제한 및 계속기업존속불확실성)',
    "'14사업연도 자본잠식률 50%이상 사유로 관리종목 지정 후 '15사업연도 반기 감사의견 거절",


    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당',
    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    '기업의 계속성, 경영의 투명성 및 기타 공익과 투자자 보호 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정',
    '기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정',
    '기업의 계속성, 경영의 투명성 또는 그 밖에 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    '기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지가 필요하다고 인정',
    '기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    '기업의계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    

    # ── 부도·거래정지 ──
    '최종부도발생',
    '최종부도 발생',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래정지',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래 정지',
    '해산사유 발생(파산선고)'
]

# ─────────────────────────────────────────────
# 2) “비-실적(자진신청·합병·지배구조·공시위반 등)” 상장폐지 사유
# ─────────────────────────────────────────────
non_performance_cols = [
    '신청에 의한 상장폐지',
    "상장폐지 신청('23.06.28)",
    "상장폐지 신청('22.08.31)",
    "상장폐지 신청('22.03.30)",
    "상장폐지 신청('22.04.14)",
    "상장폐지 신청('17.06.22)",
    "상장폐지신청('17.02.07)",
    "상장폐지신청('15.6.11)",
    "상장폐지 신청('15.01.15)",
    "상장폐지신청('14.04.22)",
    "상장폐지신청('13.09.06)",
    "상장폐지 신청('13.05.08)",
    "상장폐지신청('12.09.26)",
    "상장폐지신청('12.10.09)",
    "상장폐지 신청('12.09.26)",
    "상장폐지신청('12.09.12)",

    '지주회사(최대주주등)의 완전자회사화 등',
    '지주회사의 완전자회사화(지주회사 신규상장)',
    '해산 사유 발생',
    '존속기간 만료',
    '공시서류 미제출(사업보고서) 후 10일이내 미제출',
    'SPAC 상장예비심사청구서 미제출 등',
    '상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소',
    '상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동사유 미해소',
    '피흡수합병(스팩소멸합병)',
    '타법인의 완전자회사로 편입',
    '유가증권시장 상장',
    '피흡수합병',
    '상장예비심사청구서 미제출',
    # '상장예비심사 청구서 미제출',
    # '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동 사유를 미해소',
    '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동사유를 미해소',
    '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조제1항에 따른 사업보고서 법정제출기한 내 미제출, 최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
    '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조 제1항에 따른 분기보고서 법정기한 내 미제출',
    '법정제출기한까지 사업보고서를 제출하지 아니한 후, 법정제출기한의 다음날부터 10일이내에 사업보고서를 제출하지 아니함',
    '최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
    '사업보고서 법정제출기한 익일부터 10일 이내 미제출',
    '사업보고서 법정제출기한 익일부터 10일 이내 미제출 등',
    '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동 사유 미해소',

    # '기업의 계속성, 경영의 투명성 및 기타 공익과 투자자 보호 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    # '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정',
    # '기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정',
    # '기업의 계속성, 경영의 투명성 또는 그 밖에 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    # '기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지가 필요하다고 인정',
    # '기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    # '기업의계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정'
]

# ─────────────────────────────────────────────
# (선택) 중복·교차 점검
# ─────────────────────────────────────────────
assert set(performance_cols).isdisjoint(non_performance_cols), '중복 항목이 있습니다!'


In [76]:
performance_cols = set(performance_cols)
non_performance_cols = set(non_performance_cols)

# 모든 값이 잘 분배되었는지 확인
union_of_cols = performance_cols.union(non_performance_cols)
undistributed_from_fail = fail_set.difference(union_of_cols)
print(f"fail에 있으나 다른 컬럼에 없는 값: {undistributed_from_fail if undistributed_from_fail else '없음'}")

# performance_cols에 환각 값이 있는지 확인
hallucinated_in_performance = performance_cols.difference(fail_set)
print(f"performance_cols에 환각으로 생긴 값: {hallucinated_in_performance if hallucinated_in_performance else '없음'}")

# non_performance_cols에 환각 값이 있는지 확인
hallucinated_in_non_performance = non_performance_cols.difference(fail_set)
print(f"non_performance_cols에 환각으로 생긴 값: {hallucinated_in_non_performance if hallucinated_in_non_performance else '없음'}")

fail에 있으나 다른 컬럼에 없는 값: 없음
performance_cols에 환각으로 생긴 값: 없음
non_performance_cols에 환각으로 생긴 값: 없음


In [77]:
print(len(fail_set))
print(len(performance_cols))
print(len(non_performance_cols))

98
61
37


In [78]:
value_fail = fail[fail['폐지사유'].isin(performance_cols)]
unvalue_fail = fail[fail['폐지사유'].isin(non_performance_cols)]
value_fail.to_csv("../data/processed/value_fail.csv", index=False)
unvalue_fail.to_csv("../data/processed/unvalue_fail.csv", index=False)